# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,3.44,87,99,17.71,TF,1698710458
1,1,petropavlovsk-kamchatsky,53.0452,158.6483,4.33,62,0,5.36,RU,1698710557
2,2,vila franca do campo,37.7167,-25.4333,19.23,99,21,3.49,PT,1698710497
3,3,longyearbyen,78.2186,15.6401,0.91,64,75,3.60,SJ,1698710558
4,4,vadso,70.0744,29.7487,-2.91,86,40,4.12,NO,1698710558


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
# Create a map that displays points for every city with humidity data
map_plot = city_data_df.hvplot.points(
    x="Lng",
    y="Lat",
    c="Humidity",
    s=city_data_df["Humidity"] * 0.5,
    cmap="viridis",
    title="City Humidity"
)

# Display the map
map_plot

:Points   [Lng,Lat]   (Humidity,_size)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [6]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_city_df = city_data_df.loc[
    (city_data_df["Max Temp"] < 28) & (city_data_df["Max Temp"] > 23) \
    & (city_data_df["Wind Speed"] < 4.5) \
    & (city_data_df["Cloudiness"] == 0),:]

# Drop any rows with null values
narrowed_city_df = narrowed_city_df.dropna()

# Display sample data
narrowed_city_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
33,33,fort bragg,35.1390,-79.0060,23.91,63,0,1.54,US,1698710566
113,113,koungou,-12.7336,45.2042,26.84,83,0,1.03,YT,1698710590
268,268,matoury,4.8483,-52.3317,26.98,89,0,1.54,GF,1698710642
383,383,arraial do cabo,-22.9661,-42.0278,23.98,88,0,3.09,BR,1698710686
409,409,laguna,38.4210,-121.4238,24.05,26,0,1.54,US,1698710649


### Step 3: Create a new DataFrame called `hotel_df`.

In [8]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrowed_city_df[["City", "Lat", "Lng", "Country", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Lat,Lng,Country,Humidity,Hotel Name
33,fort bragg,35.1390,-79.0060,US,63,
113,koungou,-12.7336,45.2042,YT,83,
268,matoury,4.8483,-52.3317,GF,89,
383,arraial do cabo,-22.9661,-42.0278,BR,88,
409,laguna,38.4210,-121.4238,US,26,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [13]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories" : "accommodation.hotel",
    "apiKey" : geoapify_key,
    "limit" : 20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    print(name_address.status_code)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
200
fort bragg - nearest hotel: Airborne Inn Lodging
200
koungou - nearest hotel: Hôtel Trévani
200
matoury - nearest hotel: La Marmotte
200
arraial do cabo - nearest hotel: No hotel found
200
laguna - nearest hotel: Holiday Inn Express & Suites
200
conde - nearest hotel: Paladino
200
al kharijah - nearest hotel: Kharga Hotel
200
hassi messaoud - nearest hotel: AMC
200
yanbu - nearest hotel: No hotel found
200
dahab - nearest hotel: Lighthouse Hotel


,City,Lat,Lng,Country,Humidity,Hotel Name
33,fort bragg,35.1390,-79.0060,US,63,Airborne Inn Lodging
113,koungou,-12.7336,45.2042,YT,83,Hôtel Trévani
268,matoury,4.8483,-52.3317,GF,89,La Marmotte
383,arraial do cabo,-22.9661,-42.0278,BR,88,No hotel found
409,laguna,38.4210,-121.4238,US,26,Holiday Inn Express & Suites
459,conde,-7.2597,-34.9075,BR,83,Paladino
478,al kharijah,25.4514,30.5464,EG,55,Kharga Hotel
518,hassi messaoud,31.6804,6.0729,DZ,19,AMC
529,yanbu,24.0891,38.0637,SA,89,No hotel found
538,dahab,28.5010,34.5134,EG,58,Lighthouse Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [23]:
%%capture --no-display

# Configure the map plot and display map
map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size ="Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    title = "OSM",
    frame_width = 600,
    frame_height = 500,
    hover_cols = ["Hotel Name", "Country"]
)
map_plot

:Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)